In [1]:
import os
import gzip
import json
from collections import Counter

import h5py
from tokenizers import ByteLevelBPETokenizer
from tokenizers.trainers import BpeTrainer

from deepcode.config import parse_config
from deepcode.train.data import CodeDataset

In [2]:
os.chdir("..")

In [3]:
device = "cpu"
with open("config.toml", "r") as f:
    config = parse_config(f.read())
file = h5py.File(config.training.data_train, "r")
data = CodeDataset(file["java"], "java", device)

In [5]:
tokenizer_doc = ByteLevelBPETokenizer("../cache/vocabs/doc-vocab.json", "../cache/vocabs/doc-merges.txt")
tokenizer_java = ByteLevelBPETokenizer("../cache/vocabs/code-java-vocab.json", "../cache/vocabs/code-java-merges.txt")


In [6]:
def print_sample(idx: int):
    language, (code_tokens, code_mask, doc_tokens, doc_mask, name_tokens, name_mask) = data[idx]
    code_tokens = code_tokens[code_mask]
    print(tokenizer_java.decode_batch(list([i] for i in code_tokens)))

print_sample(0)

['protected', 'final', 'void', 'bind', 'Indexed', '(', 'Configuration', 'PropertyName', 'name', ',', 'Bind', 'able', '<', '?', '>', 'target', ',', 'Aggregate', 'Element', 'Binder', 'element', 'Binder', ',', 'Resol', 'v', 'ableType', 'aggregate', 'Type', ',', 'Resol', 'v', 'ableType', 'elementType', ',', 'Indexed', 'Collection', 'Supplier', 'result', ')', '{', 'for', '(', 'Configuration', 'Property', 'Source', 'source', ':', 'getContext', '(', ')', '.', 'getS', 'ources', '(', ')', ')', '{', 'bind', 'Indexed', '(', 'source', ',', 'name', ',', 'target', ',', 'element', 'Binder', ',', 'result', ',', 'aggregate', 'Type', ',', 'elementType', ')', ';', 'if', '(', 'result', '.', 'was', 'Sup', 'plied', '(', ')', '&&', 'result', '.', 'get', '(', ')', '!=', 'null', ')', '{', 'return', ';', '}', '}', '}']


In [5]:
with gzip.open("/Users/admin/Thesis/CodeSearchNet/resources/data/java/final/jsonl/train/java_train_0.jsonl.gz", "r") as f:
    res = [f.readline() for _ in range(5)]
res = [json.loads(s) for s in res]


In [7]:
def print_from_lines(idx: int):
    print(res[idx]["code_tokens"])

print_from_lines(0)

['protected', 'final', 'void', 'bindIndexed', '(', 'ConfigurationPropertyName', 'name', ',', 'Bindable', '<', '?', '>', 'target', ',', 'AggregateElementBinder', 'elementBinder', ',', 'ResolvableType', 'aggregateType', ',', 'ResolvableType', 'elementType', ',', 'IndexedCollectionSupplier', 'result', ')', '{', 'for', '(', 'ConfigurationPropertySource', 'source', ':', 'getContext', '(', ')', '.', 'getSources', '(', ')', ')', '{', 'bindIndexed', '(', 'source', ',', 'name', ',', 'target', ',', 'elementBinder', ',', 'result', ',', 'aggregateType', ',', 'elementType', ')', ';', 'if', '(', 'result', '.', 'wasSupplied', '(', ')', '&&', 'result', '.', 'get', '(', ')', '!=', 'null', ')', '{', 'return', ';', '}', '}', '}']


In [16]:
counter = Counter()
for i in res:
    counter += Counter(i["code_tokens"])
trainer = BpeTrainer(vocab_size = 100, min_frequency = 0)
trainer.train(counter)

AttributeError: 'BpeTrainer' object has no attribute 'train'

In [26]:
tokenizer_java.encode("\n").ids

[]